In [4]:
# IMPORTAÇÃO DE BLIBLIOTECAS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
df_consultas = pd.read_csv('consultas.csv')
df_avaliacoes = pd.read_csv('avaliacoes.csv')
df_clinicas = pd.read_csv('clinicas.csv')
df_medicos = pd.read_csv('medicos.csv')
df_pacientes = pd.read_csv('pacientes.csv')


In [15]:
# Converter colunas de data/hora para datetime
df_consultas['data_hora_agendada'] = pd.to_datetime(df_consultas['data_hora_agendada'])
df_consultas['data_hora_inicio'] = pd.to_datetime(df_consultas['data_hora_inicio'])

# Calcular tempo de espera em minutos
df_consultas['tempo_espera'] = (df_consultas['data_hora_inicio'] - df_consultas['data_hora_agendada']).dt.total_seconds() / 60
df_consultas

,id_consulta,id_paciente,id_medico,id_clinica,especialidade,data_hora_agendada,data_hora_inicio,status,tempo_espera
0,1,82,15,4,Cardiologia,2023-09-27 23:46:00,2023-09-28 00:16:00,Realizada,30.0
1,2,18,95,14,Cardiologia,2023-09-04 09:05:00,2023-09-04 13:05:00,Realizada,240.0
2,3,55,5,4,Pediatria,2023-02-04 02:43:00,2023-02-04 02:58:00,Realizada,15.0
3,4,78,4,72,Ortopedia,2023-03-14 09:28:00,2023-03-14 13:28:00,Realizada,240.0
4,5,58,76,36,Pediatria,2023-10-22 16:54:00,2023-10-22 16:54:00,Cancelada,0.0
...,...,...,...,...,...,...,...,...,...
4995,4996,27,27,93,Pediatria,2023-03-28 04:34:00,2023-03-28 05:34:00,Cancelada,60.0
4996,4997,17,26,8,Cardiologia,2023-12-06 12:39:00,2023-12-06 12:54:00,Realizada,15.0
4997,4998,60,39,34,Ortopedia,2023-11-06 11:27:00,2023-11-06 11:27:00,Realizada,0.0
4998,4999,65,7,98,Dermatologia,2023-10-07 20:19:00,2023-10-07 22:19:00,Realizada,120.0
